In [41]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [42]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

In [43]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [44]:
parameter_list2 = ['Age','SibSp','Parch','Fare', 'Pclass', 'Survived', 'PassengerId']
for par in parameter_list2:
    print('\nstats for {}\nmax :{}   min :{}\naverage : {}\nnum_nans : {}'.format(par, train_data[par].max(), train_data[par].min(), train_data[par].mean(),train_data[par].isna().astype(int).sum()))


stats for Age
max :80.0   min :0.42
average : 29.69911764705882
num_nans : 177

stats for SibSp
max :8   min :0
average : 0.5230078563411896
num_nans : 0

stats for Parch
max :6   min :0
average : 0.38159371492704824
num_nans : 0

stats for Fare
max :512.3292   min :0.0
average : 32.204207968574636
num_nans : 0

stats for Pclass
max :3   min :1
average : 2.308641975308642
num_nans : 0

stats for Survived
max :1   min :0
average : 0.3838383838383838
num_nans : 0

stats for PassengerId
max :891   min :1
average : 446.0
num_nans : 0


In [45]:
#numerical features attempt (ideas: try to use regression to predict missing age values,)
#numerical features are 'Pclass','Sex','Age','SibSp','Parch','Fare'
train_data1 = train_data.copy()
train_data1.loc[train_data1['Age'].isna(),'Age'] = train_data1['Age'].median()
train_data1['Sex'] = (train_data1['Sex'] == 'female').astype(int)
train_data1['Sex'] = train_data1['Sex']/train_data1['Sex'].max()
train_data1['Age'] = train_data1['Age']/train_data1['Age'].max()
train_data1['SibSp'] = train_data1['SibSp']/train_data1['SibSp'].max()
train_data1['Parch'] = train_data1['Parch']/train_data1['Parch'].max()
train_data1['Fare'] = train_data1['Fare']/train_data1['Fare'].max()
train_data1['Pclass'] = train_data1['Pclass']/train_data1['Pclass'].max()


X = train_data1[['Pclass','Sex','Age','SibSp','Parch','Fare']].copy()
Y = train_data1['Survived'].copy()



parameter_list = ['Pclass','Sex','Age','SibSp','Parch','Fare']

for par in parameter_list:
    print('Num nans in {} : {}'.format(par, X[par].isna().astype(int).sum()))
print('Num nans in Y : {}'.format( Y.isna().astype(int).sum()))

Num nans in Pclass : 0
Num nans in Sex : 0
Num nans in Age : 0
Num nans in SibSp : 0
Num nans in Parch : 0
Num nans in Fare : 0
Num nans in Y : 0


In [112]:
#idea introduce min threshold for a split to be valid
def DT_split(parameters,data,response_parameter):
        split_error_dict = {}
        split_impossible = True
        for param in parameters:
            temp_values = data[param].unique()
            temp_values.sort()
            if len(temp_values) > 1:
                split_impossible = False
                if len(temp_values) > 2:
                    for val in temp_values[1:-1]:
                        sum1 = (data[data[param] <= val][response_parameter].mean() - data[data[param] <= val][response_parameter])
                        sum1 = sum1*sum1
                        sum1 = sum1.sum()
                        sum2 = (data[data[param] > val][response_parameter].mean() - data[data[param] > val][response_parameter])
                        sum2 = sum2*sum2
                        sum2 = sum2.sum()
                        split_error_dict['{}-{}'.format(param,val)] =  sum1 + sum2
                else:
                    sum1 = (data[data[param] == temp_values[0]][response_parameter].mean() - data[data[param] == temp_values[0]][response_parameter])
                    sum1 = sum1*sum1
                    sum1 = sum1.sum()
                    sum2 = (data[data[param] == temp_values[1]][response_parameter].mean() - data[data[param] == temp_values[1]][response_parameter])
                    sum2 = sum2*sum2
                    sum2 = sum2.sum()
                    split_error_dict['{}-bi'.format(param)] =  sum1 + sum2
        if split_impossible:
            return [-1,'no_possible_split']
        else:
            #print(split_error_dict)
            best_split = min(split_error_dict , key = split_error_dict.get)
            return [split_error_dict[best_split],best_split]

In [116]:
def generate_tree(parameters,data,response_parameter,max_depth):
    D_tree_leaves = {'root': data}
    max_depth_notreached = True
    while max_depth_notreached:
        possible_splits = {}
        for key in D_tree_leaves:
            possible_split_temp = DT_split(parameters,D_tree_leaves[key],response_parameter)
            if possible_split_temp[0]>0:
                possible_splits[key] = possible_split_temp
        #print(possible_splits)
        if len(possible_splits) > 0:
            best_split = min(possible_splits , key = possible_splits.get)
            best_split_info = possible_splits[best_split][1]
            best_split_info_temp = best_split_info.split('-')
            if best_split_info_temp[1] == 'bi':
                temp_values = D_tree_leaves[best_split][best_split_info_temp[0]].unique()
                temp_values.sort()
                D_tree_leaves[best_split+'/'+possible_splits[best_split][1]+'_l'] = D_tree_leaves[best_split][D_tree_leaves[best_split][best_split_info_temp[0]] == float(temp_values[0])]
                D_tree_leaves[best_split+'/'+possible_splits[best_split][1]+'_r'] = D_tree_leaves[best_split][D_tree_leaves[best_split][best_split_info_temp[0]] == float(temp_values[1])]
                del D_tree_leaves[best_split]
            else:
                D_tree_leaves[best_split+'/'+possible_splits[best_split][1]+'_l'] = D_tree_leaves[best_split][D_tree_leaves[best_split][best_split_info_temp[0]] <= float(best_split_info_temp[1])]
                D_tree_leaves[best_split+'/'+possible_splits[best_split][1]+'_r'] = D_tree_leaves[best_split][D_tree_leaves[best_split][best_split_info_temp[0]] > float(best_split_info_temp[1])]
                del D_tree_leaves[best_split]
        else:
            print('max_depth cant be reached because there are no more possible splits')
            max_depth_notreached = False
            
        for key in D_tree_leaves:
            temp_key = key.split('/')
            if len(temp_key) >= max_depth:
                max_depth_notreached = False
    return D_tree_leaves
        

In [119]:
test = generate_tree(['Pclass','Sex','Age','SibSp','Parch','Fare'], train_data1, 'Survived',5)

In [120]:
test.keys()

dict_keys(['root/Sex-bi_l', 'root/Sex-bi_r/Pclass-0.6666666666666666_r', 'root/Sex-bi_r/Pclass-0.6666666666666666_l/Fare-0.05604306762136532_l', 'root/Sex-bi_r/Pclass-0.6666666666666666_l/Fare-0.05604306762136532_r/Age-0.0375_l', 'root/Sex-bi_r/Pclass-0.6666666666666666_l/Fare-0.05604306762136532_r/Age-0.0375_r'])